In [2]:
import json as j
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

In [4]:
json_data = None

with open('yelp_academic_dataset_review.json') as data_file:

    lines = data_file.readlines()
    joined_lines = "[" + ",".join(lines) + "]"

    json_data = j.loads(joined_lines)

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bitaan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
data = pd.DataFrame(json_data)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

In [8]:
data['cleaned'] = data['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned'], data.stars, test_size=0.2)

In [10]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000, dual=False))])

In [11]:
model = pipeline.fit(X_train, y_train)

vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

In [12]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [13]:
target_names = ['1', '2', '3', '4', '5']
print("top 10 keywords per class:")

top 10 keywords per class:


In [14]:
for i, label in enumerate(target_names):
    top10 = np.argsort(clf.coef_[i])[-10:]
    print("%s: %s" % (label, " ".join(feature_names[top10])))

1: disgust bogus revolt singl star place joke horribl nose dive zero star worst negat star
2: nd star particular clean respond state half experi experienc better meh experienc chase waiter turn faucet left movi second star
3: mean ok overal ok unfortun bread just expect fell littl torn place noth blew border close cool unfortun definit ok
4: lose star th star knock star margarita nice reason star took star thing star yes fan fifth star yay fan
5: guy rock woohoo good equal amaz penzey redseven amaz favor tri best worth everi stop read


In [15]:
print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.5814449132269148


In [16]:
print(model.predict(['that was an awesome place. Great food!']))

[5]
